---
---

<h1><center><ins>Exercise Sheet 7</ins></center></h1>
<h2><center>Numerical Methods <br><br>

---
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 1 - Backward Euler:

Consider the following first-order ODE:

$$ \frac{d y}{dt} = - 3 y \ , $$

**(A)** Solve this ODE with **your own forward Euler** implementation for $0\leq t\leq20$. Use the following step sizes: $\Delta t = 0.1, 0.25, 0.5, 0.75$, and plot the numerical solution for all four step sizes in four different figures. Also compare it with the exact analytical solution in the corresponding plot. Discuss your findings.

**(B)** Now implement **your own backward Euler** implementation and repeat all the steps from **(A)**. What do you notice?

**(C)** What complication arises in the implementation of the backward Euler, if we replace the right hand side of the ODE with $ (1-\frac{y}{3})y $? What kind of numerical methods already known from the lecture can you use to solve the arising equation? Please implement them to solve this ODE with the backward Euler and compare your solution with analytic one by plotting them in a figure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_solution(t, y, t_exact, y_exact, dt, method):
    plt.figure(figsize=(7,4))
    plt.plot(t, y, 'o-', label=f'{method} (dt={dt})')
    plt.plot(t_exact, y_exact, 'k--', label='Exact')
    plt.title(f'{method} method, dt={dt}')
    plt.xlabel("t")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.show()

#a) forward euler
def forward_euler(f, y0, t0, T, dt):
    N = int((T - t0) / dt)
    t = np.linspace(t0, T, N+1)
    y = np.zeros(N+1)
    y[0] = y0
    for n in range(N):
        y[n+1] = y[n] + dt * f(t[n], y[n])
    return t, y

#ODE
def f_linear(t, y):
    return -3*y

#exact solution
def exact_linear(t, y0=1):
    return np.exp(-3*t)*y0


#b) backward euler
def backward_euler_linear(y0, t0, T, dt):
    N = int((T - t0) / dt)
    t = np.linspace(t0, T, N+1)
    y = np.zeros(N+1)
    y[0] = y0
    
    #backward Euler step:
    #y_{n+1} = y_n / (1 + 3 dt)
    for n in range(N):
        y[n+1] = y[n] / (1 + 3*dt)
    return t, y



#c) backward euler for dy/dt = (1 - y/3)*y
def f_nonlinear(t, y):
    return (1 - y/3) * y

#backward euler equation
# G(Y) = Y - y_n - dt*(1 - Y/3)*Y = 0
def G(Y, yn, dt):
    return Y - yn - dt*(1 - Y/3)*Y

#derivative of newton
def dG(Y, dt):
    return 1 - dt*(1 - 2*Y/3)

#newton iteration
def newton_step(yn, dt, tol=1e-10, max_iter=50):
    Y = yn  #initial guess
    for k in range(max_iter):
        g = G(Y, yn, dt)
        dg = dG(Y, dt)
        Y_new = Y - g/dg
        if abs(Y_new - Y) < tol:
            return Y_new
        Y = Y_new
    return Y  #return best guess

#fixed-point iteration
def fixed_point_step(yn, dt, tol=1e-10, max_iter=300):
    Y = yn
    for k in range(max_iter):
        Y_new = yn + dt*(1 - Y/3)*Y
        if abs(Y_new - Y) < tol:
            return Y_new
        Y = Y_new
    return Y

#backward euler of nonlinear ODE
def backward_euler_nonlinear(y0, t0, T, dt, solver="newton"):
    N = int((T - t0) / dt)
    t = np.linspace(t0, T, N+1)
    y = np.zeros(N+1)
    y[0] = y0
    
    for n in range(N):
        if solver == "newton":
            y[n+1] = newton_step(y[n], dt)
        else:
            y[n+1] = fixed_point_step(y[n], dt)
    
    return t, y

#exact solution of y' = (1 - y/3) y:
def exact_logistic(t, y0=0.5):
    K = 3
    r = 1
    return K / (1 + ((K-y0)/y0)*np.exp(-r*t))



#execute all parts
y0 = 1
t0 = 0
T  = 20
dt_list = [0.1, 0.25, 0.5, 0.75]

#a)
for dt in dt_list:
    t, y = forward_euler(f_linear, y0, t0, T, dt)
    t_exact = np.linspace(0,20,2000)
    y_exact = exact_linear(t_exact)
    plot_solution(t, y, t_exact, y_exact, dt, method="Forward Euler")

#b)
for dt in dt_list:
    t, y = backward_euler_linear(y0, t0, T, dt)
    t_exact = np.linspace(0,20,2000)
    y_exact = exact_linear(t_exact)
    plot_solution(t, y, t_exact, y_exact, dt, method="Backward Euler")


#c)
dt = 0.1
y0 = 0.5

#newton
tN, yN = backward_euler_nonlinear(y0, 0, 20, dt, solver="newton")

#fixed point
tF, yF = backward_euler_nonlinear(y0, 0, 20, dt, solver="fixed")

#exact
t_exact = np.linspace(0,20,2000)
y_exact = exact_logistic(t_exact, y0=y0)

plt.figure(figsize=(7,4))
plt.plot(tN, yN, 'o-', label='Backward Euler (Newton)')
plt.plot(tF, yF, 's-', label='Backward Euler (Fixed point)')
plt.plot(t_exact, y_exact, 'k--', label='Exact logistic solution')
plt.title("Backward Euler for nonlinear ODE")
plt.xlabel("t")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()


## Exercise 2 - Backward Euler Part 2:

Consider now the following second-order ODE:

$$ y^{\prime\prime} + 6 y^{\prime} + 5y = 10, $$

with initial conditions of $y(0) = 0$ and $y^{\prime}(0) = 5.$

**(A)** Solve this ODE with **your own backward Euler** implementation for $0\leq t\leq5$. What is the linear system of equation that you have to solve? (write it down in matrix notation). Within **your own backward Euler** implementation you are free to solve this system of equation using either numerical methods (e.g. np.linalg.solve) or invert the matrix by hand. Use a step size of 0.1 and 0.5. Plot the absolute error of the numerical solutions for both steps sizes compared to the exact solution. The exact solution is 

$$ y(t) = -\frac{3}{4}e^{-5t} -\frac{5}{4}e^{-t} + 2. $$


**(B)** Repeat **(A)** using **forward Euler**. What do you notice?

## Exercise 3 - More Oscillators:

Consider again a system of a mass that is attached to a spring. We have seen (look at Exercise Sheet 6 Exercise Number 3) that the resuling ODE describing such a system is of the form

$$ \frac{d^2 x}{dt^2} = - \frac{k}{m} x \ , $$

where $m$ is the mass of your particle, $k$ is the spring constant, and $x$ is the displacement from the equilibrium position.

**(A)** Implement the Leapfrog integration in oder to solve this ODE. Follow the section "Algorithm" under https://en.wikipedia.org/wiki/Leapfrog_integration. Consider the same values for the constants and initial conditions from Exercise Sheet 6 and plot your result by comparing with the analytic solution. Is the accuracy the same as for the Kunge-Kutta method? Why is this integrator called leapfrog? Name one advantage of the leapfrog integration over other methods such as Kunge-Kutta.

Consider now the damped oscillator of the form:

$$ \frac{d^2 x}{dt^2} = - \frac{k}{m} x - \frac{D}{m} \frac{dx}{dt} \ , $$

where $D$ is an additional friction coefficient.

**(B)**
Solve this ODE with the Runge-Kutta method. Either use the build-in python function or your own implementation. Play around with different values for $D$ and compare it to the analytic solution (you may use : https://lemesurierb.people.charleston.edu/numerical-methods-and-analysis-python/main/ODE-IVP-4-system-higher-order-equations-python.html#equation-equation-damped-mass-spring to help you with the analytic solution). Consider all three possibilities of the system being (1) underdamped, (2) overdamped and (3) critically damped (What do these terms mean?). Make a plot for all three cases to show your numerical and analytic solution.

**(C)**
What problem arises if you were to solve the damped oscillator with the leapfrog algorithm? What could be a possible solution to that problem?